Yahoo Finance request

In [ ]:
%pip install yfinance

In [68]:
# IMPORTA BIBLIOTECA YAHOO FINACIALS

import yfinance as yf
fx = yf.Ticker("EURUSD=X")

In [84]:
# REQUISIÇAO DOS DADOS HORA/HORA

fxHist = fx.history(start="2023-03-08", end="2023-03-09", interval= "1h")

In [85]:
# RESETA INDICE E CRIA COMPARATIVO ENTRE OS RESULTADOS HORA/HORA

import pandas as pd
fxHist = fxHist.reset_index()

result = []
for i in range(0, fxHist.shape[0]):
    result.append(fxHist.iloc[i][4] / fxHist.iloc[i-1][4] -1) 

resultDf = pd.DataFrame(result)
resultNumeric = pd.to_numeric(resultDf[0])

fxHist = fxHist.join(resultNumeric, lsuffix='_caller', rsuffix='_other')
fxResult = fxHist.rename(columns={0: 'result'})

In [ ]:
fxResult

In [ ]:
fxDict = fxHist.to_json(orient='index') ##convertendo resultado em json
print(fxDict)